In [296]:
#Importa a biblioteca do Gurobi
import gurobipy as gp

In [297]:
def le_dados(nome_arq):
    with open(nome_arq, "r") as f:
        linhas = f.readlines()
        valores = linhas[0].strip().split(" ")
        energiaNecessaria = int(valores[0])
        proteinaNecessaria = int(valores[1])
        calciumNecessaria = int(valores[2])
        totalAlimentos = int(valores[3])
        # Vetores de Food Energy Protein Calcium Price Min Max
        vet_comida = list()
        vet_energia = list()
        vet_proteina = list()
        vet_calcium= list()
        vet_preco = list()
        vet_min = list()
        vet_max = list()
        del(linhas[0])
        for linha in linhas:
            valores = linha.strip().split(" ")
            vet_comida.append(valores[0])
            vet_energia.append(int(valores[1]))
            vet_proteina.append(int(valores[2]))
            vet_calcium.append(int(valores[3]))
            vet_preco.append(int(valores[4]))
            vet_min.append(int(valores[5]))
            vet_max.append(int(valores[6]))
        #rotulos de comida
        comidas=list()
        for i in range(totalAlimentos):
            rotulo = vet_comida[i]
            comidas.append(rotulo)
        # Cria o dicionário de energia
        energias = dict()
        for idx, energia in enumerate(vet_energia):
            rotulo = comidas[idx]
            energias[rotulo] = energia
        # Cria o dicionário de proteina
        proteinas = dict()
        for idx, proteina in enumerate(vet_proteina):
            rotulo = comidas[idx]
            proteinas[rotulo] = proteina
        # Cria o dicionário de calcium
        calciums = dict()
        for idx,calcium in enumerate(vet_calcium):
            rotulo = comidas[idx]
            calciums[rotulo] = calcium
        # Cria o dicionário de preco
        precos = dict()
        for idx,preco in enumerate(vet_preco):
            rotulo = comidas[idx]
            precos[rotulo] = preco
        # Cria o dicionário de mim
        mins = dict()
        for idx,min in enumerate(vet_min):
            rotulo = comidas[idx]
            mins[rotulo] = min
        # Cria o dicionário de max
        maxs = dict()
        for idx,max in enumerate(vet_max):
            rotulo = comidas[idx]
            maxs[rotulo] = max
            
    # Retorna os dados lidos
        return energiaNecessaria,proteinaNecessaria,calciumNecessaria,totalAlimentos,comidas,energias,proteinas,calciums,precos,mins,maxs

In [329]:
def solver_diet(nome_arq):
    #Le dados do arquivo
    energiaNecessaria,proteinaNecessaria,calciumNecessaria,totalAlimentos,comidas,energias,proteinas,calciums,precos,mins,maxs = le_dados(nome_arq)
    #criar modelo
    m = gp.Model()
    m.setParam(gp.GRB.Param.OutputFlag,0) #mudando flag para omitir dados

    #Inserir as variaveis de decisão (6)
    x = m.addVars(comidas, vtype=gp.GRB.INTEGER) #metodo adiciona as variaveis da lista de comidas , além disso como especificado no problema elas são inteiras

    #Funcao objetivo, deixando modelo mais flexivel
    m.setObjective(
        gp.quicksum(x[i] * precos[i] for i in comidas),#gerando uma tupla e somando(somatorio i=1 até o total de comidas somando o produto (xi * precos))
        sense = gp.GRB.MINIMIZE
    )
    
    #Restrição de mins
    
    cmins=m.addConstrs((mins[i] <= x[i]) for i in comidas)
    
    #Restrição de maxs
    cmaxs=m.addConstrs((x[i] <= maxs[i]) for i in comidas)
    
    #Restrição de energia
    m.addConstr(gp.quicksum(x[i] * energias[i] for i in comidas) >= energiaNecessaria)
    
    #Restrição de proteina
    m.addConstr(gp.quicksum(x[i] * proteinas[i] for i in comidas) >= proteinaNecessaria)
    
    #Restrição de calcium
    m.addConstr(gp.quicksum(x[i] * calciums[i] for i in comidas) >= calciumNecessaria)

    #Exececuta o modelo
    m.optimize()
    
    #Printa resultado
    lista_valores = list()
    for comida in comidas:
           lista_valores.append(x[comida].X)

    lista_nome = list()
    for y in range (0,totalAlimentos):
           print(comidas[y],lista_valores[y])
           lista_nome.append(comidas[y])


In [332]:
arq_padrao = "InstanciasDieta/inst_{:03d}.txt" #numero inteiro com 3 digitos
for i in range(2) :
    print("------------------------------------------------------------------")
    nome_arq = arq_padrao.format(i)
    solver_diet(nome_arq)
    print("------------------------------------------------------------------")

------------------------------------------------------------------
Oatmeal 3.0
Chicken -0.0
Eggs -0.0
WholeMilk 8.0
CherryPie 1.0
PorkBeans -0.0
------------------------------------------------------------------
------------------------------------------------------------------
Oatmeal 2.0
Chicken -0.0
Eggs -0.0
WholeMilk 7.0
CherryPie 1.0
PorkBeans -0.0
Beans 1.0
------------------------------------------------------------------


arq_padrao = "InstanciasDieta/inst_000.txt"
solver_diet(arq_padrao)


In [ ]:
arq_padrao = "InstanciasMochila/inst_{:03d}.txt" #numero inteiro com 3 digitos
for i in range(100) :
    nome_arq = arq_padrao.format(i)
    objetivo,lista,capacidade = solver_mochila(nome_arq)
    print("------------------------------------------------------------------")
    print(" Instancia : ",i," Valor objetivo : ",objetivo," Capacidade utillizada : ",capacidade)
    print(lista)
    print("------------------------------------------------------------------")

In [23]:
#Criar o modelo
m=gp.Model("Problema da Dieta")
#Insere as variaveis de decisão
x1 = m.addVar(vtype=gp.GRB.INTEGER) # Oatmeal
x2 = m.addVar(vtype=gp.GRB.INTEGER) # Chicken
x3 = m.addVar(vtype=gp.GRB.INTEGER) # Eggs
x4 = m.addVar(vtype=gp.GRB.INTEGER) # Whole Milk
x5 = m.addVar(vtype=gp.GRB.INTEGER) # Cherry Pie
x6 = m.addVar(vtype=gp.GRB.INTEGER) # Pork with Beans
#Definir função objetivo
m.setObjective(3 * x1 + 24 * x2 + 13 * x3 + 9 * x4 + 20 * x5 + 19 * x6, sense=gp.GRB.MINIMIZE) #Queremos pagar o menor valor possível
#Inserir as restrições do modelo
c1 = m.addConstr(0 <= x1)  # Até 4 refeições
c11 = m.addConstr(x1 <= 4)  # Até 4 refeições
c2 = m.addConstr(0 <= x2)  # Até 3 refeições
c22 = m.addConstr(x2 <= 3)  # Até 3 refeições
c3 = m.addConstr(0 <= x3)  # Até 2 refeições
c33 = m.addConstr(x3 <= 2)  # Até 2 refeições
c4 = m.addConstr(0 <= x4)  # Até 8 refeições
c44 = m.addConstr(x4 <= 8)  # Até 8 refeições
c5 = m.addConstr(0 <= x5)  # Até 2 refeições
c55 = m.addConstr(x5 <= 2)  # Até 2 refeições
c6 = m.addConstr(0 <= x6)  # Até 2 refeições
c66 = m.addConstr(x6 <= 2)  # Até 2 refeições
c7 = m.addConstr((110 * x1 + 205 * x2 + 160 * x3 + 160 * x4 + 420 * x5 + 260 * x6) >=2000)  # Energia em kcal
c8 = m.addConstr((4 * x1 + 32 * x2 + 13 * x3 + 8 * x4 + 4 * x5 + 14 * x6) >=55)  # Proteina
c9 = m.addConstr((2 * x1 + 12 * x2 + 54 * x3 + 285 * x4 + 22 * x5 + 80 * x6) >=2000)  # Calcium
#Resolver o modelo
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 15 rows, 6 columns and 30 nonzeros
Model fingerprint: 0x60c244b3
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [3e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+03]
Presolve removed 13 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 6 columns, 12 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)
Found heuristic solution: objective 120.0000000

Root relaxation: objective 9.582234e+01, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   95.82234    0    2  120.00000   95.82234  20.1%     -    0s
H    0     0                     108.0000000   95.82234  11.3%     -    0s

In [25]:
# Imprimir o quantidade

print("Oatmeal :", x1.X )
print("Chicken :", x2.X )
print("Eggs :", x3.X )
print("Whole Milk :", x4.X )
print("Cherry Pie :", x5.X )
print("Pork with Beans :", x6.X )

Oatmeal : 3.0
Chicken : -0.0
Eggs : -0.0
Whole Milk : 8.0
Cherry Pie : 1.0
Pork with Beans : -0.0
